# Project Description
This project is focused on the Prediction of Concrete compressive strength(MPa, megapascals) based different commonents
The dependent Variable is Concrete compressive strength(MPa, megapascals), while the independent variables are:
1) Cement (component 1)(kg in a m^3 mixture)

2) Blast Furnace Slag (component 2)(kg in a m^3 mixture)	
3) Fly Ash (component 3)(kg in a m^3) 

4) Water (component 4)(kg in a m^3 mixture)	
5) Superplasticizer (component 5)(kg in a m^3 mixture)	
6) Coarse Aggregate (component 6)(kg in a m^3 mixture)	
7) Fine Aggregate (component 7)(kg in a m^3 mixture)	

Different LightGBM models will be developed based on different age of curing.


# Import Important Libraries

In [ ]:
import pandas as pd
import fast_ml
from sklearn.preprocessing  import OneHotEncoder
from fast_ml import eda
from sklearn.model_selection import train_test_split
import os
from sklearn.linear_model import Ridge, Lasso
from sklearn import metrics
import numpy as np
from sklearn.metrics import r2_score
import math
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import hydroeval as he
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
%matplotlib inline

# Data Preparation and Assessment

In [ ]:
#Load and View Data
filepath='Concrete_Data.xlsx'
df=pd.read_excel(filepath,engine='openpyxl') #without engine='openpyxl', I was unable to read the excel file) 
df.head(5)

In [ ]:
from fast_ml import eda

# One of the most useful dataframe summary view
summary_df = eda.df_info(df)
summary_df

In [ ]:
#simple Descriptive Statistics of Data
df.describe(include='all').transpose()

In [ ]:
corr = df.corr()
print("\nCorrelation matrix:")
print(corr)

# Heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap of Correlation Matrix', fontsize=16)
plt.show()


In [ ]:
# Histogram for numerical columns
df.hist(bins=20, figsize=(14, 10), color='skyblue')
plt.suptitle('Histograms of Numerical Columns', fontsize=16)
plt.show()
